# Assignment Set 2
by Zahraoruji for Deep Learing Course at Data Science Center, SBU

In [1]:
import numpy as np 
import pandas as pd
import torch
import torchvision
import torchvision.transforms as transforms

# Augmentation without Regularization

In [22]:
transform = transforms.Compose([
     transforms.Resize((200, 200)),
     transforms.CenterCrop((100, 100)),
     transforms.RandomCrop((32, 32)),
     transforms.RandomHorizontalFlip(p=0.5),
     transforms.RandomRotation(degrees=(-90, 90)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
     ])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=4,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=4,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [23]:
import torch.nn as nn
import torch.nn.functional as F


class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 6, 5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 16 * 5 * 5)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x


net = Net()
net

Net(
  (conv1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  (fc1): Linear(in_features=400, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [24]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [25]:
LAMBDA = 0.009

for epoch in range(2):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        
        
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 2.295
[1,  4000] loss: 2.192
[1,  6000] loss: 2.140
[1,  8000] loss: 2.127
[1, 10000] loss: 2.105
[1, 12000] loss: 2.083
[2,  2000] loss: 2.080
[2,  4000] loss: 2.069
[2,  6000] loss: 2.070
[2,  8000] loss: 2.067
[2, 10000] loss: 2.057
[2, 12000] loss: 2.060
Finished Training


In [26]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane :  0 %
Accuracy of   car : 55 %
Accuracy of  bird : 14 %
Accuracy of   cat :  1 %
Accuracy of  deer : 16 %
Accuracy of   dog : 25 %
Accuracy of  frog : 57 %
Accuracy of horse :  8 %
Accuracy of  ship : 23 %
Accuracy of truck : 13 %


# Augmentation with Regularization

In [27]:
LAMBDA = 0.009

for epoch in range(2):

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        
         #L1 regularization
        
        l2_regularization = torch.tensor(0).long()
        for param in net.parameters():
            l2_regularization += torch.norm(param, 1).long()
            
        loss += LAMBDA * l2_regularization
        
        
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

[1,  2000] loss: 19.863
[1,  4000] loss: 19.944
[1,  6000] loss: 20.023
[1,  8000] loss: 20.099
[1, 10000] loss: 20.201
[1, 12000] loss: 20.284
[2,  2000] loss: 20.386
[2,  4000] loss: 20.460
[2,  6000] loss: 20.548
[2,  8000] loss: 20.626
[2, 10000] loss: 20.711
[2, 12000] loss: 20.781
Finished Training


In [28]:
class_correct = list(0. for i in range(10))
class_total = list(0. for i in range(10))
with torch.no_grad():
    for data in testloader:
        images, labels = data
        outputs = net(images)
        _, predicted = torch.max(outputs, 1)
        c = (predicted == labels).squeeze()
        for i in range(4):
            label = labels[i]
            class_correct[label] += c[i].item()
            class_total[label] += 1


for i in range(10):
    print('Accuracy of %5s : %2d %%' % (
        classes[i], 100 * class_correct[i] / class_total[i]))

Accuracy of plane : 19 %
Accuracy of   car :  9 %
Accuracy of  bird :  5 %
Accuracy of   cat : 18 %
Accuracy of  deer :  8 %
Accuracy of   dog : 18 %
Accuracy of  frog : 51 %
Accuracy of horse : 38 %
Accuracy of  ship : 53 %
Accuracy of truck : 19 %
